In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.special import logit, expit
import statsmodels.formula.api as smf
import statsmodels.api as sm
from collections import defaultdict
from sklearn.metrics import brier_score_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
pd.set_option('display.max_columns', 100)

In [36]:
df = pd.read_csv('soccer18m.csv', index_col = False)
df2a = df

In [32]:
df2a['Market_Home'] = (df2a.pH - df2a.pA  > 0).astype(int)
df2a['Market_Away'] = (df2a.pA - df2a.pH > 0).astype(int)
df2a['Market_Difference'] = np.abs(df2a.pH - df2a.pA)
df2a['Home_win'] = (df2a['FTHG'] - df2a['FTAG'] > 0).astype(int)
df2a['Away_win'] = (df2a['FTAG'] - df2a['FTHG'] > 0).astype(int)
df2a['Expected_Result'] = (df2a.Home_win == df2a.Market_Home) | (df2a.Away_win == df2a.Market_Away)

,Div,Date,Y,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,pH,pD,pA,home_xG,away_xG,Market_Home,Market_Away,Market_Difference,Home_win,Away_win,Expected_Result
0,EPL,2014-08-16,14,Arsenal,Crystal Palace,2,1,1,1,14,4,6,2,0.778038,0.151989,0.069973,1.554110,0.158151,1,0,0.708064,1,0,True
1,EPL,2014-08-16,14,Leicester,Everton,2,2,1,2,11,13,3,3,0.311972,0.289820,0.398208,1.278300,0.613273,0,1,0.086236,0,0,True
2,EPL,2014-08-16,14,Man United,Swansea,1,2,0,1,14,5,5,4,0.715373,0.192169,0.092459,1.166350,0.278076,1,0,0.622914,0,1,False
3,EPL,2014-08-16,14,QPR,Hull,0,1,0,0,19,11,6,4,0.395113,0.300577,0.304311,1.900670,1.117570,1,0,0.090802,0,1,False
4,EPL,2014-08-16,14,Stoke,Aston Villa,0,1,0,0,12,7,2,2,0.502378,0.282316,0.215305,0.423368,0.909774,1,0,0.287073,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9125,Ligue_1,2018-12-18,18,Caen,Toulouse,2,1,1,1,13,7,7,2,0.383840,0.311487,0.304673,1.447980,1.385310,1,0,0.079167,1,0,True
9126,Ligue_1,2018-12-23,18,Bordeaux,Amiens,1,1,1,0,9,8,4,4,0.599829,0.250444,0.149726,0.477339,0.348658,1,0,0.450103,0,0,True
9127,Ligue_1,2019-01-27,18,Toulouse,Angers,0,0,0,0,14,13,4,3,0.423877,0.291891,0.284232,1.462170,1.339530,1,0,0.139645,0,0,True
9128,Ligue_1,2019-01-30,18,Nantes,St Etienne,1,1,0,0,13,10,3,4,0.346213,0.305100,0.348686,1.654040,1.829150,0,1,0.002473,0,0,True


In [33]:
df2a = df2a.loc[df['Y'] < 18]
df2a = df2a.loc[df2a['Expected_Result'] == False]
df2a = df2a.nlargest(7, 'Market_Difference')
df2a = df2a[['Div','Y','HomeTeam','AwayTeam','FTHG','FTAG','pH','pA']]
df2a

,Div,Y,HomeTeam,AwayTeam,FTHG,FTAG,pH,pA
2677,La_Liga,16,Barcelona,Alaves,1,2,0.891147,0.028831
2128,La_Liga,14,Barcelona,Malaga,0,1,0.875453,0.040021
3081,La_Liga,17,Real Madrid,Betis,0,1,0.876513,0.048646
1988,La_Liga,14,Barcelona,Celta,0,1,0.861781,0.043664
4291,Bundesliga,15,Bayern Munich,Mainz,1,2,0.856920,0.044404
2602,La_Liga,15,Barcelona,Valencia,1,2,0.837164,0.056947
8688,Serie_A,17,Inter,Sassuolo,1,2,0.833965,0.057824


In [37]:
df = df.drop(['pD'], axis=1)
df['GD_Home'] = df.FTHG - df.FTAG
df['GD_Away'] = df.FTAG - df.FTHG
df['Pts_Home'] = np.where(df['FTHG'] == df['FTAG'], 1, np.where( df['FTHG'] >  df['FTAG'], 3, 0))
df['Pts_Away'] = np.where(df['FTAG'] == df['FTHG'], 1, np.where( df['FTAG'] >  df['FTHG'], 3, 0))
df['xPts_Home'] = np.where(round(df['home_xG']) == round(df['away_xG']), 1, np.where( round(df['home_xG']) >  round(df['away_xG']), 3, 0))
df['xPts_Away'] = np.where(round(df['away_xG']) == round(df['home_xG']), 1, np.where( round(df['away_xG']) >  round(df['home_xG']), 3, 0))
df['Conversion_Home'] = df.FTHG/df.home_xG
df['Conversion_Away'] = df.FTAG/df.away_xG
df['Home_win'] = (df['FTHG'] - df['FTAG'] > 0).astype(int)
df['Market_Home'] = (df.pH - df.pA  > 0).astype(int)
df['Market_Away'] = (df.pA - df.pH > 0).astype(int)
df = df.rename({'HomeTeam':'Team_Home','AwayTeam':'Team_Away','FTHG':'FTG_Home','FTAG':'FTG_Away', 
               'HTHG':'HTG_Home','HTAG':'HTG_Away', 'HS':'S_Home','AS':'S_Away', 'HST':'ST_Home','AST':'ST_Away',
               'home_xG':'xG_Home','away_xG':'xG_Away', 'pH':'p_Home', 'pA':'p_Away'}, axis = 1)
df.loc[:,'GameID'] = df.index
df = pd.wide_to_long(df, ['Team','FTG','HTG','S','ST','xG','GD','Pts','xPts','Conversion','Market','p'], i = ['GameID'], j = 'isHome', sep = '_', suffix = r'\w+')
df = df.reset_index().sort_values(['Date','GameID'])
df.loc[:,'isHome'] = 1*(df.isHome == 'Home')
df

,GameID,isHome,Home_win,Date,Div,Y,Team,FTG,HTG,S,ST,xG,GD,Pts,xPts,Conversion,Market,p
5306,5306,1,0,2014-08-08,Ligue_1,14,Reims,2,2,9,3,1.367870,0,1,0,1.462127,0,0.089841
14436,5306,0,0,2014-08-08,Ligue_1,14,Paris SG,2,1,16,6,2.655380,0,1,3,0.753188,1,0.713484
5307,5307,1,0,2014-08-09,Ligue_1,14,Bastia,3,1,13,4,1.396890,0,1,1,2.147628,0,0.245226
14437,5307,0,0,2014-08-09,Ligue_1,14,Marseille,3,2,9,4,1.317510,0,1,1,2.277023,1,0.462559
5308,5308,1,0,2014-08-09,Ligue_1,14,Evian Thonon Gaillard,0,0,10,2,0.813737,-3,0,1,0.000000,1,0.399734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18213,9083,0,1,2019-05-26,Serie_A,18,Juventus,0,0,6,1,0.487175,-2,0,0,0.000000,1,0.505747
9084,9084,1,0,2019-05-26,Serie_A,18,Spal,2,1,7,4,0.433984,-1,0,0,4.608465,0,0.155177
18214,9084,0,0,2019-05-26,Serie_A,18,Milan,3,2,16,8,1.784600,1,3,3,1.681049,1,0.629777
9085,9085,1,1,2019-05-26,Serie_A,18,Torino,3,0,9,7,2.398040,2,3,3,1.251022,1,0.416710


In [72]:
df.loc[:,'GP'] = df.groupby('Team').Pts.transform(lambda x : x.rolling(len(df), 1).count().shift(1, fill_value = 0))
df['GP_Season'] = (df['GP']) % np.where(df['Div'] == 'Bundesliga', 34, 38)
df.loc[:,'G_total'] = df.groupby('Team').FTG.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'Pts_total'] = df.groupby('Team').Pts.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'Form3'] = df.groupby(['Team','Y']).Pts.transform(lambda x : x.rolling(3, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form5'] = df.groupby(['Team','Y']).Pts.transform(lambda x : x.rolling(5, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form7'] = df.groupby(['Team','Y']).Pts.transform(lambda x : x.rolling(7, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form3g'] = df.groupby(['Team','Y']).FTG.transform(lambda x : x.rolling(3, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form5g'] = df.groupby(['Team','Y']).FTG.transform(lambda x : x.rolling(5, 1).sum().shift(1, fill_value = 0))
df.loc[:,'Form7g'] = df.groupby(['Team','Y']).FTG.transform(lambda x : x.rolling(7, 1).sum().shift(1, fill_value = 0))
df.loc[:,'G_season'] = df.groupby(['Y','Team']).FTG.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'Pts_season'] = df.groupby(['Y','Team']).Pts.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'Rank'] = df.groupby(['Y','Div', 'GP_Season'])['Pts_season'].rank("average", ascending=False)
df.loc[:,'GD_total'] = df.groupby('Team').GD.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'xG_total'] = df.groupby('Team').xG.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'xPts_total'] = df.groupby(['Team']).xPts.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'xPts_season'] = df.groupby(['Y','Team']).xPts.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'xRank'] = df.groupby(['Y','Div', 'GP_Season'])['xPts_season'].rank("average", ascending=False)
df.loc[:,'market_win'] = df.groupby('Team').p.transform(lambda x : x.cumsum().shift(1, fill_value = 0))
df.loc[:,'market_win_form5'] = df.groupby(['Team', 'Y']).p.transform(lambda x : x.rolling(5, 1).sum().shift(1, fill_value = 0))
df.loc[:,'gp_form5'] = df.groupby(['Team', 'Y']).p.transform(lambda x : x.rolling(5, 1).count().shift(1, fill_value = 0))
df['AGD'] = df['GD_total']/df['GP']
df['AxG'] = df['xG_total']/df['GP']
df['Amw'] = df['market_win']/df['GP']
df['AmwF'] = df['market_win_form5']/df['gp_form5']
df['Reason'] = np.where((df['Rank'] >= 15) & (df['GP_Season'] >= 30), 1, 0)
df = df.fillna(0)
df

,GameID,isHome,Home_win,Date,Div,Y,Team,FTG,HTG,S,ST,xG,GD,Pts,xPts,Conversion,Market,p,GP,GP_Season,G_total,Pts_total,Form3,Form5,Form7,Form3g,Form5g,Form7g,G_season,Pts_season,Rank,GD_total,xG_total,xPts_total,xPts_season,xRank,AGD,AxG,Reason,market_win,Amw,market_win_form5,gp_form5,AmwF
5306,5306,1,0,2014-08-08,Ligue_1,14,Reims,2,2,9,3,1.367870,0,1,0,1.462127,0,0.089841,0,0,0,0,0,0,0,0,0,0,0,0,10.5,0,0.000000,0,0,10.5,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,0.000000
14436,5306,0,0,2014-08-08,Ligue_1,14,Paris SG,2,1,16,6,2.655380,0,1,3,0.753188,1,0.713484,0,0,0,0,0,0,0,0,0,0,0,0,10.5,0,0.000000,0,0,10.5,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,0.000000
5307,5307,1,0,2014-08-09,Ligue_1,14,Bastia,3,1,13,4,1.396890,0,1,1,2.147628,0,0.245226,0,0,0,0,0,0,0,0,0,0,0,0,10.5,0,0.000000,0,0,10.5,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,0.000000
14437,5307,0,0,2014-08-09,Ligue_1,14,Marseille,3,2,9,4,1.317510,0,1,1,2.277023,1,0.462559,0,0,0,0,0,0,0,0,0,0,0,0,10.5,0,0.000000,0,0,10.5,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,0.000000
5308,5308,1,0,2014-08-09,Ligue_1,14,Evian Thonon Gaillard,0,0,10,2,0.813737,-3,0,1,0.000000,1,0.399734,0,0,0,0,0,0,0,0,0,0,0,0,10.5,0,0.000000,0,0,10.5,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18213,9083,0,1,2019-05-26,Serie_A,18,Juventus,0,0,6,1,0.487175,-2,0,0,0.000000,1,0.505747,189,37,380,454,2,6,9,2,5,8,70,90,1.0,257,314.071637,423,76,2.0,1.359788,1.661755,0,122.170867,0.646407,2.311374,5.0,0.462275
9084,9084,1,0,2019-05-26,Serie_A,18,Spal,2,1,7,4,0.433984,-1,0,0,4.608465,0,0.155177,75,37,81,80,3,7,10,7,12,15,42,42,11.0,-31,77.463586,70,39,13.0,-0.413333,1.032848,0,18.907945,0.252106,1.330208,5.0,0.266042
18214,9084,0,0,2019-05-26,Serie_A,18,Milan,3,2,16,8,1.784600,1,3,3,1.681049,1,0.629777,189,37,270,301,9,10,13,5,6,8,52,65,5.0,56,261.907505,285,59,7.0,0.296296,1.385754,0,86.847150,0.459509,2.722955,5.0,0.544591
9085,9085,1,1,2019-05-26,Serie_A,18,Torino,3,0,9,7,2.398040,2,3,3,1.251022,1,0.416710,189,37,274,266,4,10,12,5,8,9,49,60,7.0,26,249.261002,243,45,10.0,0.137566,1.318841,0,72.001387,0.380960,1.810047,5.0,0.362009


In [73]:
df1 = df.loc[(df.isHome == 1)]
df2 = df.loc[(df.isHome == 0)]
dfmerge = pd.merge(df1, df2, on=('GameID', 'Y', 'Date','Home_win','Div'), how='outer', suffixes=('_Home', '_Away'))
dfmerge.tail(6)

,GameID,isHome_Home,Home_win,Date,Div,Y,Team_Home,FTG_Home,HTG_Home,S_Home,ST_Home,xG_Home,GD_Home,Pts_Home,xPts_Home,Conversion_Home,Market_Home,p_Home,GP_Home,GP_Season_Home,G_total_Home,Pts_total_Home,Form3_Home,Form5_Home,Form7_Home,Form3g_Home,Form5g_Home,Form7g_Home,G_season_Home,Pts_season_Home,Rank_Home,GD_total_Home,xG_total_Home,xPts_total_Home,xPts_season_Home,xRank_Home,AGD_Home,AxG_Home,Reason_Home,market_win_Home,Amw_Home,market_win_form5_Home,gp_form5_Home,AmwF_Home,isHome_Away,Team_Away,FTG_Away,HTG_Away,S_Away,ST_Away,xG_Away,GD_Away,Pts_Away,xPts_Away,Conversion_Away,Market_Away,p_Away,GP_Away,GP_Season_Away,G_total_Away,Pts_total_Away,Form3_Away,Form5_Away,Form7_Away,Form3g_Away,Form5g_Away,Form7g_Away,G_season_Away,Pts_season_Away,Rank_Away,GD_total_Away,xG_total_Away,xPts_total_Away,xPts_season_Away,xRank_Away,AGD_Away,AxG_Away,Reason_Away,market_win_Away,Amw_Away,market_win_form5_Away,gp_form5_Away,AmwF_Away
9124,9080,1,0,2019-05-26,Serie_A,18,Fiorentina,0,0,5,5,0.296039,0,1,1,0.000000,1,0.383114,189,37,285,285,0,0,1,0,1,1,47,40,15.5,49,276.947506,301,57,8.0,0.259259,1.465331,1,84.595258,0.447594,1.744490,5.0,0.348898,0,Genoa,0,0,2,1,0.074391,0,1,1,0.000000,0,0.216357,189,37,217,219,2,3,4,3,4,5,39,37,18.0,-42,234.476966,221,39,13.0,-0.222222,1.240619,1,60.184504,0.318437,1.636270,5.0,0.327254
9125,9081,1,1,2019-05-26,Serie_A,18,Inter,2,0,20,15,2.743020,1,3,3,0.729123,1,0.695512,189,37,302,322,4,6,10,3,5,8,55,66,3.5,105,304.790265,336,75,3.0,0.555556,1.612647,0,97.436332,0.515536,2.681817,5.0,0.536363,0,Empoli,1,0,9,5,1.772750,-1,0,0,0.564095,0,0.123785,151,37,165,158,9,9,10,7,10,12,50,38,17.0,-65,172.881224,151,35,15.0,-0.430464,1.144909,1,43.705572,0.289441,1.794867,5.0,0.358973
9126,9082,1,1,2019-05-26,Serie_A,18,Roma,2,1,16,8,1.722810,1,3,3,1.160894,1,0.803027,189,37,352,377,5,9,15,3,7,9,64,63,6.0,167,330.576535,371,67,5.0,0.883598,1.749082,0,104.221513,0.551437,2.359049,5.0,0.471810,0,Parma,1,0,9,5,1.089380,-1,0,0,0.917953,0,0.068488,75,37,73,67,4,6,8,5,7,7,40,41,13.0,-61,71.775209,52,32,16.0,-0.813333,0.957003,0,18.292597,0.243901,1.339976,5.0,0.267995
9127,9083,1,1,2019-05-26,Serie_A,18,Sampdoria,2,0,10,3,0.686933,2,3,3,2.911492,0,0.249003,189,37,259,248,2,2,5,4,5,7,58,50,9.0,-10,239.932721,215,50,9.0,-0.052910,1.269485,0,61.729024,0.326609,1.885401,5.0,0.377080,0,Juventus,0,0,6,1,0.487175,-2,0,0,0.000000,1,0.505747,189,37,380,454,2,6,9,2,5,8,70,90,1.0,257,314.071637,423,76,2.0,1.359788,1.661755,0,122.170867,0.646407,2.311374,5.0,0.462275
9128,9084,1,0,2019-05-26,Serie_A,18,Spal,2,1,7,4,0.433984,-1,0,0,4.608465,0,0.155177,75,37,81,80,3,7,10,7,12,15,42,42,11.0,-31,77.463586,70,39,13.0,-0.413333,1.032848,0,18.907945,0.252106,1.330208,5.0,0.266042,0,Milan,3,2,16,8,1.784600,1,3,3,1.681049,1,0.629777,189,37,270,301,9,10,13,5,6,8,52,65,5.0,56,261.907505,285,59,7.0,0.296296,1.385754,0,86.847150,0.459509,2.722955,5.0,0.544591
9129,9085,1,1,2019-05-26,Serie_A,18,Torino,3,0,9,7,2.398040,2,3,3,1.251022,1,0.416710,189,37,274,266,4,10,12,5,8,9,49,60,7.0,26,249.261002,243,45,10.0,0.137566,1.318841,0,72.001387,0.380960,1.810047,5.0,0.362009,0,Lazio,1,0,9,4,1.179200,-2,0,0,0.848033,0,0.323954,189,37,341,324,4,7,10,6,9,11,55,59,8.0,108,309.183868,324,61,6.0,0.571429,1.635893,0,92.479280,0.489308,2.589093,5.0,0.517819


In [74]:
df2b = dfmerge.loc[dfmerge['Y'] < 18]
result = smf.glm('Home_win ~ p_Home', data = df2b, family = sm.families.Binomial()).fit()
print(brier_score_loss(df2b.Home_win, result.predict(df2b)))

df2b['Early_Season'] = (df2b.GP_Season_Home < 5) & (df2b.GP_Season_Away < 5)
df2be = df2b.loc[df2b['Early_Season'] == True]
resulte = smf.glm('Home_win ~ p_Home', data = df2be, family = sm.families.Binomial()).fit()
print(brier_score_loss(df2be.Home_win, resulte.predict(df2be)))

0.21073632061847292
0.21062737443541976


<ipython-input-74-af7f272b3f00>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2b['Early_Season'] = (df2b.GP_Season_Home < 5) & (df2b.GP_Season_Away < 5)


In [91]:
dfhw = dfmerge.loc[dfmerge['Y'] < 18]
df18 = dfmerge.loc[dfmerge['Y'] == 18]
result = smf.glm('Home_win ~ AGD_Away + Form5g_Home + Form5g_Away + Rank_Home + xRank_Away + Reason_Home + Amw_Home + AmwF_Away', data = dfhw, family = sm.families.Binomial()).fit()
result.summary(), print(brier_score_loss(df18.Home_win, result.predict(df18)))



0.21220171528698126


(<class 'statsmodels.iolib.summary.Summary'>
 """
                  Generalized Linear Model Regression Results                  
 Dep. Variable:               Home_win   No. Observations:                 7304
 Model:                            GLM   Df Residuals:                     7295
 Model Family:                Binomial   Df Model:                            8
 Link Function:                  logit   Scale:                          1.0000
 Method:                          IRLS   Log-Likelihood:                -4509.4
 Date:                Wed, 24 Feb 2021   Deviance:                       9018.8
 Time:                        11:53:29   Pearson chi2:                 7.26e+03
 No. Iterations:                     4                                         
 Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
 --------------------------------------------------------------------